In [ ]:
import os 
from dotenv import load_dotenv

load_dotenv()

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage,HumanMessage

In [3]:
loader = PyPDFLoader("attention.pdf")
text = loader.load()

In [4]:
docs = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents = docs.split_documents(text)

In [5]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

In [6]:
db = FAISS.from_documents(documents,embeddings)
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026144B1E7D0>, search_kwargs={})

In [7]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7
)

In [8]:
history_prompt = ChatPromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {input}
Standalone Question:"""
)

chat_retriever_chain = create_history_aware_retriever(
    llm, retriever, history_prompt
)

In [9]:
system_prompt =     """Your task is to answer the following question based on the following context provided to you

    and always try to answer based on thecontext if you dont know the answer just say I dont no
    
    <context>
    {context}
    </context>
    
    question: {input}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [10]:
llm_chain = create_stuff_documents_chain(llm,qa_prompt)
rag_chain = create_retrieval_chain(chat_retriever_chain,llm_chain)

In [11]:
chat_history = []

question="Who are the autthors of research paper attention is all you need?"
response=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response["answer"])
    ]
)
response

{'input': 'Who are the autthors of research paper attention is all you need?',
 'chat_history': [HumanMessage(content='Who are the autthors of research paper attention is all you need?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The authors of the research paper "Attention is All You Need" are: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin.', additional_kwargs={}, response_metadata={})],
 'context': [Document(id='bdd8ab91-62dc-4cfa-8bca-b172fa939a17', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-03T00:07:29+00:00', 'author': '', 'keywords': '', 'moddate': '2023-08-03T00:07:29+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'attention.pdf', 'total_pages': 15, 'page': 2, 'page_label': '3'}, page_content='3.2

In [12]:
question="Total how many where they?"
response=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response["answer"])
    ]
)
response

{'input': 'Total how many where they?',
 'chat_history': [HumanMessage(content='Who are the autthors of research paper attention is all you need?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The authors of the research paper "Attention is All You Need" are: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Total how many where they?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='There were eight authors in total.', additional_kwargs={}, response_metadata={})],
 'context': [Document(id='bdd8ab91-62dc-4cfa-8bca-b172fa939a17', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-03T00:07:29+00:00', 'author': '', 'keywords': '', 'moddate': '2023-08-03T00:07:29+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpaths

In [13]:
question="Name all of them"
response=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response["answer"])
    ]
)
response

{'input': 'Name all of them',
 'chat_history': [HumanMessage(content='Who are the autthors of research paper attention is all you need?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The authors of the research paper "Attention is All You Need" are: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Total how many where they?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='There were eight authors in total.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Name all of them', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin.', additional_kwargs={}, response_metadata={})],
 'context': [Document(id='60e56e33-4b6a-4a53-9100-558fc1bfbdd2', metadat